# Simulation: Passing Percentage

In [1]:
import os, sys, random, statistics
import pandas as PD

In [2]:
BACKEND_PATH = os.path.dirname(os.path.dirname(os.getcwd()))
APPPROJ_PATH = os.path.join(BACKEND_PATH, 'app_proj')
sys.path.append(APPPROJ_PATH)
import notebooks as NT
print([x for x in dir(NT) if x[0]!='_' and len(x)>8])

['DataframeToDicts', 'GetRandom', 'NOTEBOOK_ENV']


In [3]:
MODULE_PATH = os.path.dirname(os.getcwd())
DATA_PATH = os.path.join(MODULE_PATH, 'data')
os.path.exists(DATA_PATH)

True

In [4]:
import django
os.environ['DEPLOYMENT_ENV'] = NT.NOTEBOOK_ENV
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'
os.chdir(os.path.dirname(MODULE_PATH))
django.setup()

env: DEV


In [5]:
import emporium.logic.stage as ST
print([x for x in dir(ST) if x[0]!='_' and len(x)>8])

['AssembleRoom', 'CheckPermitted', 'DIFFICULTY', 'GetDevAgility', 'GetDevCombat', 'GetDevCunning', 'GetDevMight', 'GetDevelopmentTable', 'GetExpedition', 'GetHealAmount', 'GetMagicItem', 'GetRareMaterial', 'GetStageRewards', 'GetStarThief', 'GetTreasureReward', 'LandingType', 'ObstacleSequence', 'ProductionTable', 'RollDamage', 'RollReward', 'StageBackground']


In [6]:
import emporium.logic.simulation as SM
print([x for x in dir(SM) if x[0]!='_' and len(x)>8])

['ApplyRandomLevels', 'AttachWargear', 'RunBeatCount', 'RunPassTest', 'TestExpedition']


In [7]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import IPython.display as DS 
from ipywidgets import IntProgress

### Simulation

In [143]:
baseThief = {
    'Agi': 3, 'Cun': 0, 'Mig': 0, 'End': 0,
    'Hlt': 80,
    'Sab': 0, 'Per': 0, 'Tra': 0, 
}
# baseThief['Mig'] += 2
baseThief = SM.AttachWargear(baseThief)
baseThief['Hlt'] += 6
PD.DataFrame([baseThief])

,Agi,Cun,Mig,End,Hlt,Sab,Per,Tra,Att,Dmg,Def
0,5,0,0,0,86,0,0,0,5,7,14


In [144]:
THIEF_LEVEL = 0
thiefConfig = SM.ApplyRandomLevels(baseThief, THIEF_LEVEL)
PD.DataFrame([thiefConfig])

,Agi,Cun,Mig,End,Hlt,Sab,Per,Tra,Att,Dmg,Def
0,5,0,0,0,86,0,0,0,5,7,14


In [145]:
MAX_OBSTACLES = 12
STAGE_TYPE = 'balanced'   #'biased agi'
ENEMY_LEVEL = 1

potentialLs = ST.ProductionTable(ENEMY_LEVEL,2,1,1,1)
obstacleLs = ST.ObstacleSequence(potentialLs, MAX_OBSTACLES)
permitted = ST.CheckPermitted(obstacleLs, STAGE_TYPE, MAX_OBSTACLES)

while not permitted:
    obstacleLs = ST.ObstacleSequence(potentialLs, MAX_OBSTACLES)
    permitted = ST.CheckPermitted(obstacleLs, STAGE_TYPE, MAX_OBSTACLES)

PD.DataFrame(obstacleLs)[0:1]

,World,Level,Name,Trait,Skill,Success,Failure,Experience,Attack,Damage,Defense,Health,Difficulty
0,1,1,Vanguard,All,Fight,"experience, pass",knockout,16,2.0,8,13.0,11.0,NaN


In [146]:
runLs = SM.RunPassTest(thiefConfig, obstacleLs)
PD.DataFrame(runLs)[-1:]

,obstacle,passed,curr pos,next pos,wounds,xp,gold
9,Spike Trap,False,11,12,28,7,20


In [147]:
TRIALS = 90000
clearedLs = []
progressBar = IntProgress(min=0, max=TRIALS/10000) 
DS.display(progressBar) 

for tr in range(0, TRIALS):
    if tr % 10000 == 0: progressBar.value += 1

    thiefConfig = SM.ApplyRandomLevels(baseThief, THIEF_LEVEL)
    
    obstacleLs = ST.ObstacleSequence(potentialLs, MAX_OBSTACLES)
    permitted = ST.CheckPermitted(obstacleLs, STAGE_TYPE, MAX_OBSTACLES)

    while not permitted:
        obstacleLs = ST.ObstacleSequence(potentialLs, MAX_OBSTACLES)
        permitted = ST.CheckPermitted(obstacleLs, STAGE_TYPE, MAX_OBSTACLES)

    runLs = SM.RunPassTest(thiefConfig, obstacleLs)
    clearedLs.append( runLs[-1]['next pos'] == MAX_OBSTACLES )

round(statistics.mean(clearedLs)*100, 1)

IntProgress(value=0, max=9)

85.4